In [5]:
import pickle
from collections import defaultdict
import pandas as pd
import sqlite3
import csv
import numpy as np
import os

In [28]:
directory = '../raw_data/audio_features_all/' #get directory of artist data
extract = defaultdict(list)   
for file in os.listdir(directory): #for each txt file in folder
    openFile = open(directory+file, 'rb') #open txt file
    loadFile = pickle.load(openFile) #load pickle object
    try:
        for audiofile in loadFile:
            if audiofile is None:
                extract = extract
            else:
                for key in audiofile:
                    extract[str(key)].append(audiofile[key]) #append to dictionary
    except:
        print(file)

In [36]:
df_audio = pd.DataFrame.from_dict(extract)
df_audio = df_audio.drop(labels = ['track_href', 'analysis_url', 'type'], axis = 1) #drop unuseful columns
df_audio.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,duration_ms,time_signature
0,0.777,0.552,11,-10.937,0,0.0413,0.373,0.092400,0.146,0.928,100.043,5t3qH3xZNxTscMT0adF8hF,spotify:track:5t3qH3xZNxTscMT0adF8hF,323082,4
1,0.760,0.449,0,-12.164,1,0.0760,0.150,0.001200,0.535,0.507,116.980,29sUjTxYtYEg2ATTwfzM2N,spotify:track:29sUjTxYtYEg2ATTwfzM2N,284500,4
2,0.591,0.646,9,-8.926,0,0.0799,0.013,0.000018,0.459,0.530,84.050,2vXwyfoHIHSCFCExJVjf9S,spotify:track:2vXwyfoHIHSCFCExJVjf9S,229900,4
3,0.884,0.600,1,-7.878,1,0.3150,0.141,0.013100,0.261,0.504,121.984,3iNe4HN91zPEIZVJgmn9CE,spotify:track:3iNe4HN91zPEIZVJgmn9CE,277358,4
4,0.815,0.765,7,-6.543,1,0.1950,0.275,0.000000,0.323,0.445,120.008,1zw5c8a4nI0xdPOPqgeBJQ,spotify:track:1zw5c8a4nI0xdPOPqgeBJQ,226268,4


In [37]:
df_audio.shape

(2399969, 15)

In [38]:
df_audio.to_csv('../raw_data/track_info.csv') #save as track_info.csv

In [39]:
df_audio.columns #check columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'id', 'uri', 'duration_ms', 'time_signature'],
      dtype='object')

In [40]:
#connect to database
db = sqlite3.connect('../raw_data/spotify.db')
cursor = db.cursor()
# cursor.execute("DROP TABLE IF EXISTS audio_features_first28750") #drop previous created table

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

In [41]:
#prep new table
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))
#create a new table for features_partial
cursor.execute('''CREATE TABLE IF NOT EXISTS track_info
               (Track_uri TEXT, duration_ms, acousticness REAL, danceability, 
                energy, explicit, instrumentalness, key, liveness, loudness, mode, 
                speechiness, tempo, valence, time_signature)''')

In [42]:
#write information from track_info.csv into database
with open('../raw_data/track_info.csv') as file:
    info = csv.reader(file, delimiter = ',')
    next(info, None)
    for row in info:
        index, danceability, energy, key, loudness, mode, speechiness, acousticness, intrumentalness, liveness, valence, tempo, idx, uri, duration_ms, time_signature = row
        #append to database, append explicit = 0 as there is no information on explicitness
        insert_into_db = (uri, duration_ms, acousticness, danceability, energy, 0, intrumentalness, key, liveness, loudness, mode, speechiness, tempo, valence, time_signature)
        cursor.execute('''INSERT INTO track_info
                         (Track_uri, duration_ms, acousticness, danceability, 
                          energy, explicit,instrumentalness, key, liveness, loudness, mode, 
                          speechiness, tempo, valence, time_signature) 
                          VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', insert_into_db)

In [43]:
track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track_info)")] #get columns of track_info table

In [44]:
#check track_info
query = '''SELECT * FROM track_info LIMIT 10'''
viz_tables(track_cols, query)

,Track_uri,duration_ms,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,time_signature
0,spotify:track:5t3qH3xZNxTscMT0adF8hF,323082,0.3730,0.777,0.552,0,0.0924,11,0.146,-10.937,0,0.0413,100.043,0.928,4
1,spotify:track:29sUjTxYtYEg2ATTwfzM2N,284500,0.1500,0.76,0.449,0,0.0012,0,0.535,-12.164,1,0.076,116.98,0.507,4
2,spotify:track:2vXwyfoHIHSCFCExJVjf9S,229900,0.0130,0.591,0.646,0,1.82e-05,9,0.459,-8.926,0,0.0799,84.05,0.53,4
3,spotify:track:3iNe4HN91zPEIZVJgmn9CE,277358,0.1410,0.884,0.6,0,0.0131,1,0.261,-7.878,1,0.315,121.984,0.504,4
4,spotify:track:1zw5c8a4nI0xdPOPqgeBJQ,226268,0.2750,0.815,0.765,0,0.0,7,0.323,-6.543,1,0.195,120.008,0.445,4
5,spotify:track:0upn2c2d9jGu6nI6hiTjW9,379229,0.0141,0.9,0.583,0,0.0411,11,0.269,-8.936,0,0.0436,126.041,0.951,4
6,spotify:track:3izKzu6DvM9DuyT1W7MCJ9,483825,0.1200,0.833,0.541,0,0.687,6,0.0903,-13.532,1,0.0709,105.014,0.16,4
7,spotify:track:58kQrWFK5D6wCImGBwKASp,231968,0.0068,0.797,0.91,0,0.501,10,0.0327,-5.03,0,0.0434,116.995,0.935,4
8,spotify:track:57OIIo8EOCxiMHXLeHHI5z,279158,0.0591,0.738,0.495,0,0.0188,10,0.351,-4.025,0,0.0715,94.998,0.449,4
9,spotify:track:2mSrVZsSKn2Ye5f0YuAbMh,140360,0.7870,0.579,0.43,0,0.0,5,0.13,-10.021,1,0.344,123.999,0.538,4


In [45]:
#commit to database
db.commit()
db.close()